In [104]:
import json
import mysql.connector as mysql
from datetime import date
import datetime

mysql_config_file = "./config/saraSqlConfig.json"
    
with open(mysql_config_file) as f:
    mysql_connect_object = json.load(f)

db = mysql.connect(
    host = mysql_connect_object["host"],
    port = mysql_connect_object["port"],
    user = mysql_connect_object["user"],
    passwd = mysql_connect_object["passwd"],
    database = mysql_connect_object["database"]
)

username = "mash_aya"
cursor = db.cursor()
select_statement = "SELECT * FROM harvardSurvey where user_id='" + username + "' order by survey_completion_time desc;"
cursor.execute(select_statement)
survey_data = cursor.fetchall()
survey_completion_dates = []
for x in survey_data:
    survey_completion_dates.append(x[-2].split(" ")[0])
db.close()




In [105]:
# get the 
number_of_days= 15
adherence_record_vals = []
adherence_record_date = []

for i in range(number_of_days):
    prior_date = datetime.datetime.now() - datetime.timedelta(number_of_days-1-i)
    
    prior_date_str = prior_date.strftime("%Y-%m-%d")
    
    adherence_record_date.append(prior_date_str + " 12:00 am")
    
    if prior_date_str in survey_completion_dates:
        adherence_record_vals.append(1)
    else:
        adherence_record_vals.append(0.5)
        
# make correction if there is no date before this date.

In [106]:
import pandas as pd
survey_adherence_data = {
    'date': adherence_record_date,
    'survey_completed': adherence_record_vals  
}

#-- convert to pandas dataframe, with time based indexing
survey_adherence_data_frame = pd.DataFrame(survey_adherence_data, columns = ['date','survey_completed']) 

In [107]:
survey_adherence_data_frame

,date,survey_completed
0,2020-12-17 12:00 am,0.5
1,2020-12-18 12:00 am,0.5
2,2020-12-19 12:00 am,0.5
3,2020-12-20 12:00 am,0.5
4,2020-12-21 12:00 am,1.0
5,2020-12-22 12:00 am,0.5
6,2020-12-23 12:00 am,1.0
7,2020-12-24 12:00 am,0.5
8,2020-12-25 12:00 am,0.5
9,2020-12-26 12:00 am,0.5


In [111]:
import altair as alt
user_id = 'mash_aya'
alt.Chart(survey_adherence_data_frame).mark_bar(size=10).encode(
    x=alt.X('date:T', axis=alt.Axis(format='%m/%d', title='day', labelAngle=-45, grid=False)),
    y=alt.Y('survey_completed:Q', axis=alt.Axis(labels=False)),
    color=alt.condition(
        alt.datum.survey_completed > 0.5,
        alt.value("#06982d"),  # The positive color
        alt.value("#ae1325")  # The negative color
    )
).properties(
    width=500,
    height=100,
    title=alt.TitleParams(
        ['For ' + user_id],
        baseline='bottom',
        orient='bottom',
        anchor='end',
        fontWeight='normal',
        fontSize=10
    )
)

alt.Chart(...)